In [ ]:
# Reading tables from MySQL database and converting them to pandas dataframe
import mysql.connector as connection

class Database:
    
    def __init__(self):
        self.temp = dict()
        try:
            # Creating connection to MySQL server
            host = input("Enter host name: ") or ""
            database = input("Enter database name: ") or ""
            user = input("Enter Username: ") or ""
            passwd = input("Enter Password: ") or ""
            self.mydb = connection.connect(host = host, database = database, user = user, passwd = passwd, use_pure=True)
            query1 = "show tables;" # Query to select all table names
            _tables = pd.read_sql(query1,self.mydb).iloc[:, 0].to_list() #List of tables in Database

            #Iterating on table names to create respective pandas dataframe
            for i in _tables:
                query2 = "Select * from {}".format(i) # Query to read all data from tables
                globals()['%s'%i] = pd.read_sql(query2,self.mydb) # Using dynamic variables to store each tables in respective pandas dataframe
#                 globals()['%s'%i] = globals()['%s'%i].dropna(axis=1,how='all') # Removes null columns from tables
#                 globals()['%s'%i].drop(columns=globals()['%s'%i].columns[globals()['%s'%i].nunique()==1], inplace=True) # Removes columns with single unique value
#                 for j in globals()['%s'%i].columns.values.tolist(): # Iterate on columns of dataframe
#                     try:
#                         globals()['%s'%i][j].replace(to_replace=[None], value=0, inplace=True) # Replace None value with 0
#                         globals()['%s'%i][j] = globals()['%s'%i][j].astype('int') # Convert datatype from object to int, of columns having all integer values 
#                     except:
#                         pass
                self.temp[i] = globals()['%s'%i] # Storing name and dataframe as key value pair in temp
            self.mydb.close() #close the connection

        except Exception as e:
            self.mydb.close()
            print(str(e))
            
    def empty_tables(self):
        empty_df = list()
        for k,v in self.temp.items():
            if(len(v.index) < 1):
                empty_df.append(k)
        print("There are {} empty tables \n".format(len(empty_df)))
        return empty_df
    
    def tables(self):
        x = int(input("You want tables having rows more than? : ") or '5')
        df = list()
        z = input("Do you want to print tables? Press 'y for yes and 'n' for no") or 'n'
        for k,v in self.temp.items():
            if(len(v.index) >= x):
                df.append(k)
                if(z == 'y'):
                    print(k)
                    print(v.head())
                    print('-'*120)
        print("There are {} tables with rows more than {}".format(len(df), x))
        return df
    
    def exact_tables(self):
        e_df = list()
        x = int(input('How many rows? : ') or '1')
        for k,v in self.temp.items():
            if(len(v.index) == x):
                e_df.append(k)
        print("There are {} tables with rows eual to {}".format(len(e_df),x))
        return e_df
    
    def unique_columns(self):
        u_df = list()
        for k,v in self.temp.items():
            if(len(v.index) >= 5):
                print(k)
                u_df.append(k)
                columns = v.columns.values.tolist()
                for i in columns:
                    print(i+": "+str(len(v[i].unique())))
                print("\n")